In [1]:
!pip install llama-index
!pip install llama-index-llms-mistralai
!pip install llama-index-embeddings-mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 40.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.4/187.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.0/375.0 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.9.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("MISTRAL_API_KEY")
os.environ['MISTRAL_API_KEY'] = secret_value_0

In [3]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.mistralai import MistralAI
from llama_index.embeddings.mistralai import MistralAIEmbedding
from llama_index.core import Settings

from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors.llm_selectors import LLMSingleSelector

In [5]:
llm = MistralAI(model='mistral-large-latest') 
embed_model = MistralAIEmbedding()

Settings.llm = llm
Settings.embed_model = embed_model

In [6]:
!wget "https://constitutioncenter.org/media/files/constitution.pdf" -q -O ./us_constitution.pdf
!wget "https://www.constituteproject.org/constitution/United_Kingdom_2013.pdf" -q -O ./uk_constitution.pdf

In [7]:
us_docs = SimpleDirectoryReader(input_files=["./us_constitution.pdf"]).load_data()

uk_docs = SimpleDirectoryReader(input_files=["./uk_constitution.pdf"]).load_data()

In [8]:
us_index = VectorStoreIndex.from_documents(us_docs)

uk_index = VectorStoreIndex.from_documents(uk_docs)

In [9]:
us_query_engine = us_index.as_query_engine(similarity_top_k=5)

uk_query_engine = uk_index.as_query_engine(similarity_top_k=5)

In [10]:
from llama_index.core.query_engine import CustomQueryEngine

class LLMQueryEngine(CustomQueryEngine):
    """RAG String Query Engine."""

    llm: llm

    def custom_query(self, query_str: str):

        response = self.llm.complete(query_str)

        return str(response)

llm_query_engine = LLMQueryEngine(llm=llm)

In [11]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=us_query_engine,
        metadata=ToolMetadata(
            name="us_query_engine",
            description="US Constitution",
        ),
    ),
    QueryEngineTool(
        query_engine=uk_query_engine,
        metadata=ToolMetadata(
            name="uk_query_engine",
            description="UK Constitution",
        ),
    )
]

In [12]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=True,
)
agent = AgentRunner(agent_worker)

In [13]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=us_query_engine,
        metadata=ToolMetadata(
            name="us_form",
            description="Queries related to US Constitution.",
        ),
    ),
    QueryEngineTool(
        query_engine=uk_query_engine,
        metadata=ToolMetadata(
            name="uk_form",
            description="Queries related to UK Constitution.",
        ),
    ),
    QueryEngineTool(
        query_engine=agent,
        metadata=ToolMetadata(
            name="us_uk_form",
            description=(
                "Queries related to US and UK Constitution."

            )
        )
    ),
    QueryEngineTool(
        query_engine=llm_query_engine,
        metadata=ToolMetadata(
            name="general_queries",
            description=(
                "Provides information about general queries other than constitutions."
            )
        )
    )
]

In [14]:
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=query_engine_tools,
    verbose = True
)

In [15]:
response = query_engine.query("What is the capital of Switzerland?")
display(response.response)

Selecting query engine 3: The question 'What is the capital of Switzerland?' is not related to the US or UK Constitution, so it falls under general queries..


"The capital of Switzerland is Bern. It's known for its well-preserved Old Town, which is a UNESCO World Heritage site. However, Switzerland's largest city is Zürich, which is often mistakenly thought of as the capital due to its size and economic importance."

In [16]:
response = query_engine.query("What color is the US flag?")
display(response.response)

Selecting query engine 3: The question is about the color of the US flag, which is not related to the US or UK Constitution..


'The flag of the United States features three colors:\n\n1. **Red**: There are seven red stripes.\n2. **White**: There are six white stripes, and the stars are white.\n3. **Blue**: The rectangle in the upper left corner (known as the canton or union) is blue and contains the white stars.\n\nThe combination of these colors and elements creates the distinctive look of the U.S. flag.'

In [17]:
response = query_engine.query("What is the first ammendment in the US Constitution?")
display(response.response)

Selecting query engine 0: The question specifically asks about the first amendment in the US Constitution, which falls under queries related to the US Constitution..


'The first amendment in the US Constitution states that Congress shall make no law respecting an establishment of religion, or prohibiting the free exercise thereof; or abridging the freedom of speech, or of the press; or the right of the people peaceably to assemble, and to petition the Government for a redress of grievances.'

In [18]:
response = query_engine.query("What is habeas corpus in the UK Constitution?")
display(response.response)

Selecting query engine 1: The question specifically asks about 'habeas corpus' in the context of the UK Constitution..


"Habeas corpus is a legal procedure in the UK Constitution that safeguards individual liberty. It is a writ that requires a person under arrest to be brought before a judge or into court, especially to secure the person's release unless lawful grounds are shown for their detention. The Habeas Corpus Act of 1679 outlines the specifics of this procedure, ensuring that individuals are not unjustly imprisoned or detained without due process."

In [19]:
response = query_engine.query("Which laws in the US and UK Constitution grant the right to trail by jury?")
display(response.response)

Selecting query engine 2: The question specifically asks about laws in both the US and UK Constitutions, which makes choice (3) the most relevant..
Added user message to memory: Which laws in the US and UK Constitution grant the right to trail by jury?
=== Calling Function ===
Calling function: us_query_engine with args: {"input": "Which laws grant the right to trail by jury?"}
=== Function Output ===
The right to trial by jury is granted in several places:

1. **Article III, Section 2**: It states that the trial of all crimes, except in cases of impeachment, shall be by jury.
2. **Amendment VI**: In all criminal prosecutions, the accused shall enjoy the right to a speedy and public trial, by an impartial jury.
3. **Amendment VII**: In suits at common law, where the value in controversy shall exceed twenty dollars, the right of trial by jury shall be preserved.
=== Calling Function ===
Calling function: uk_query_engine with args: {"input": "Which laws grant the right to trail by jury?"

"In the US Constitution, the right to trial by jury is granted in several places:\n\n1. **Article III, Section 2**: It states that the trial of all crimes, except in cases of impeachment, shall be by jury.\n2. **Amendment VI**: In all criminal prosecutions, the accused shall enjoy the right to a speedy and public trial, by an impartial jury.\n3. **Amendment VII**: In suits at common law, where the value in controversy shall exceed twenty dollars, the right of trial by jury shall be preserved.\n\nIn the UK Constitution, the right to trial by jury is granted in certain circumstances as outlined in the provisions of the law. Specifically, an action in the Queen's Bench Division must be tried with a jury if it involves a charge of fraud, a claim in respect of malicious prosecution or false imprisonment, or any question or issue of a kind prescribed for this purpose. Additionally, the court has discretion to order a trial with a jury in other cases."

In [20]:
response = query_engine.query("Which laws grant right to freedom of speech/expression in US and UK Constitution?")
display(response.response)

Selecting query engine 2: The question specifically asks about laws granting the right to freedom of speech/expression in both the US and UK Constitutions..
Added user message to memory: Which laws grant right to freedom of speech/expression in US and UK Constitution?
=== Calling Function ===
Calling function: us_query_engine with args: {"input": "Which laws grant right to freedom of speech/expression?"}
=== Function Output ===
The right to freedom of speech and expression is granted by the First Amendment of the Constitution.
=== Calling Function ===
Calling function: uk_query_engine with args: {"input": "Which laws grant right to freedom of speech/expression?"}
=== Function Output ===
The right to freedom of speech and expression is granted by several provisions. One key provision states that freedom of speech and debates or proceedings in Parliament ought not to be impeached or questioned in any court or place outside of Parliament. Additionally, Article 10 of the European Conventio

'The right to freedom of speech and expression is granted by the First Amendment of the Constitution in the US.\n\nIn the UK, the right to freedom of speech and expression is granted by several provisions. One key provision states that freedom of speech and debates or proceedings in Parliament ought not to be impeached or questioned in any court or place outside of Parliament. Additionally, Article 10 of the European Convention on Human Rights, which is incorporated into UK law, guarantees the right to freedom of expression, including the freedom to hold opinions and to receive and impart information and ideas without interference by public authority.'

In [21]:
response = query_engine.query("What is the most powerful position in the US and UK?")
display(response.response)

Selecting query engine 2: The question asks about the most powerful position in both the US and UK, which relates to queries about both the US and UK Constitutions..
Added user message to memory: What is the most powerful position in the US and UK?
=== Calling Function ===
Calling function: us_query_engine with args: {"input": "What is the most powerful position in the US?"}
=== Function Output ===
The President is the most powerful position in the US, serving as both the head of state and the head of the government. The President is the Commander in Chief of the armed forces and has the power to execute laws, make treaties, and appoint key officials.
=== Calling Function ===
Calling function: uk_query_engine with args: {"input": "What is the most powerful position in the UK?"}
=== Function Output ===
The most powerful position in the UK is the Head of State, which is held by the monarch.
=== LLM Response ===
The most powerful position in the US is the President, who serves as both the

'The most powerful position in the US is the President, who serves as both the head of state and the head of the government. The President is the Commander in Chief of the armed forces and has the power to execute laws, make treaties, and appoint key officials.\n\nIn the UK, the most powerful position is the Head of State, which is held by the monarch.'

In [22]:
response = query_engine.query("Which Constitution banned capital punishment?")
display(response.response)

Selecting query engine 2: The question asks about a specific aspect that could be related to either the US or UK Constitution, making choice 3 the most relevant..
Added user message to memory: Which Constitution banned capital punishment?
=== Calling Function ===
Calling function: us_query_engine with args: {"input": "Which Constitution banned capital punishment?"}
=== Function Output ===
The provided context does not mention any constitution banning capital punishment.
=== Calling Function ===
Calling function: uk_query_engine with args: {"input": "Which Constitution banned capital punishment?"}
=== Function Output ===
The Thirteenth Protocol to the Convention, agreed at Vilnius on 3rd May 2002, concerns the abolition of the death penalty in all circumstances.
=== LLM Response ===
The UK Constitution banned capital punishment.


'The UK Constitution banned capital punishment.'

In [23]:
response = query_engine.query("Who makes the laws in a country?")
display(response.response)

Selecting query engine 3: The question 'Who makes the laws in a country?' is a general query about lawmaking and does not specifically relate to the US or UK Constitution..


"In most countries, laws are made through a process involving several branches of government, primarily the legislative branch. Here's a simplified breakdown:\n\n1. **Legislative Branch**: This is the primary law-making body in most countries. It's often called Parliament, Congress, or National Assembly, depending on the country. The legislative branch is usually made up of elected representatives who propose, debate, and vote on bills. A bill becomes a law once it's passed by the legislature and receives any necessary approval from other branches of government.\n\n   - In the United States, for example, Congress (consisting of the House of Representatives and the Senate) makes federal laws.\n   - In the United Kingdom, Parliament (consisting of the House of Commons and the House of Lords) makes laws.\n\n2. **Executive Branch**: In many countries, the executive branch, led by a president or prime minister, also plays a role in law-making. The executive can often propose laws, and in so

In [24]:
response = query_engine.query("Who makes the laws in the US and UK Constitution?")
display(response.response)

Selecting query engine 2: The question asks about lawmakers in both the US and UK Constitutions, which corresponds to queries related to both constitutions..
Added user message to memory: Who makes the laws in the US and UK Constitution?
=== Calling Function ===
Calling function: us_query_engine with args: {"input": "Who makes the laws?"}
=== Function Output ===
Congress makes the laws.
=== Calling Function ===
Calling function: uk_query_engine with args: {"input": "Who makes the laws?"}
=== Function Output ===
The Assembly makes the laws, which are known as Acts of the Assembly.
=== LLM Response ===
In the US Constitution, Congress makes the laws. In the UK Constitution, the Assembly makes the laws, which are known as Acts of the Assembly.


'In the US Constitution, Congress makes the laws. In the UK Constitution, the Assembly makes the laws, which are known as Acts of the Assembly.'